Run project_clean_city_with_paranthesis first. That is specific to city names with paranthesis. It also adds a country field.

In [1]:
import pandas as pd
import numpy as np
import time
import logging
import nltk
import re
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
!pip install contractions
import contractions
!pip install country_list
import country_list

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [24]:
countries = dict(country_list.countries_for_language('en'))
countries

{'AF': 'Afghanistan',
 'AX': 'Åland Islands',
 'AL': 'Albania',
 'DZ': 'Algeria',
 'AS': 'American Samoa',
 'AD': 'Andorra',
 'AO': 'Angola',
 'AI': 'Anguilla',
 'AQ': 'Antarctica',
 'AG': 'Antigua & Barbuda',
 'AR': 'Argentina',
 'AM': 'Armenia',
 'AW': 'Aruba',
 'AU': 'Australia',
 'AT': 'Austria',
 'AZ': 'Azerbaijan',
 'BS': 'Bahamas',
 'BH': 'Bahrain',
 'BD': 'Bangladesh',
 'BB': 'Barbados',
 'BY': 'Belarus',
 'BE': 'Belgium',
 'BZ': 'Belize',
 'BJ': 'Benin',
 'BM': 'Bermuda',
 'BT': 'Bhutan',
 'BO': 'Bolivia',
 'BA': 'Bosnia & Herzegovina',
 'BW': 'Botswana',
 'BV': 'Bouvet Island',
 'BR': 'Brazil',
 'IO': 'British Indian Ocean Territory',
 'VG': 'British Virgin Islands',
 'BN': 'Brunei',
 'BG': 'Bulgaria',
 'BF': 'Burkina Faso',
 'BI': 'Burundi',
 'KH': 'Cambodia',
 'CM': 'Cameroon',
 'CA': 'Canada',
 'CV': 'Cape Verde',
 'BQ': 'Caribbean Netherlands',
 'KY': 'Cayman Islands',
 'CF': 'Central African Republic',
 'TD': 'Chad',
 'CL': 'Chile',
 'CN': 'China',
 'CX': 'Christmas Isla

In [2]:
cities_file = "us_cities_states_counties.csv"
cities_df = pd.read_csv(cities_file, delimiter="|")
city_list = cities_df.City.unique().tolist()

cities_df["State short"].unique()

array(['NY', 'PR', 'VI', 'MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NJ', 'AE',
       'PA', 'DE', 'DC', 'VA', 'MD', 'WV', 'NC', 'SC', 'GA', 'FL', nan,
       'AL', 'TN', 'MS', 'KY', 'OH', 'IN', 'MI', 'IA', 'WI', 'MN', 'SD',
       'ND', 'MT', 'IL', 'MO', 'KS', 'NE', 'LA', 'AR', 'OK', 'TX', 'CO',
       'WY', 'ID', 'UT', 'AZ', 'NM', 'NV', 'CA', 'AP', 'HI', 'AS', 'GU',
       'PW', 'FM', 'MP', 'MH', 'OR', 'WA', 'AK'], dtype=object)

In [3]:
cleaned_file_name = "sightings_cleaned.pkl"
sightings_cleaned = pd.read_pickle(cleaned_file_name)

In [4]:
len(sightings_cleaned)

67841

In [5]:
sightings_cleaned.head(30)

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State
0,4/23/21 06:30,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...,strangei recorded video sighting,NaN,NaN,Blackshear,GA
1,4/23/21 06:00,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...,ball skyobject appears white ball vapor strewi...,NaN,NaN,Mechanicsville,VA
2,4/23/21 06:00,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...,driving saw something strange sky pulled car i...,NaN,NaN,Vero Beach,FL
3,4/23/21 05:59,Light,3 minutes,2 extremely bright lights appeared over east c...,4/23/21,http://www.nuforc.org/webreports/162/S162824.html,\n2 extremely bright lights appeared over east...,2 extremely bright light appeared east coast n...,NaN,NaN,St. Augustine,FL
4,4/23/21 05:58,Cone,>5 minutes,A cone of light coming from the sky unlike any...,4/23/21,http://www.nuforc.org/webreports/162/S162819.html,\nA cone of light coming from the sky unlike a...,cone light coming sky unlike anything ever see...,NaN,NaN,Durham,NC
5,4/23/21 05:55,Sphere,10 minutes,Noticed a intense light that was covering a la...,4/23/21,http://www.nuforc.org/webreports/162/S162823.html,\nDriving on I-16 south and noticed a intense ...,driving i-16 south noticed intense light cover...,NaN,NaN,I-16 south,GA
6,4/23/21 05:54,Light,5 minutes,Two bright lights one flashing with a descendi...,4/23/21,http://www.nuforc.org/webreports/162/S162820.html,\nTwo bright lights one flashing with a descen...,two bright light one flashing descending expan...,NaN,NaN,Parrish,FL
7,4/23/21 05:45,Light,~10-15 minutes,Im former military and have never seen aircraf...,4/23/21,http://www.nuforc.org/webreports/162/S162826.html,\nIm former military and have never seen aircr...,I former military never seen aircraft that.inc...,NaN,NaN,Champions Gate,FL
9,4/23/21 02:40,Chevron,3-4 seconds,"I witnessed a chevron-shaped object, silent an...",4/23/21,http://www.nuforc.org/webreports/162/S162827.html,"\nI witnessed a chevron-shaped object, silent ...","witnessed chevron-shaped object , silent seven...",NaN,NaN,Firestone,CO
10,4/22/21 22:23,Fireball,2 minutes,Saw a steady pulsating fireball above that mov...,4/23/21,http://www.nuforc.org/webreports/162/S162818.html,\nSaw a steady pulsating fireball above that m...,saw steady pulsating fireball moved slowly awa...,New York City (Brooklyn),United States,New York,NY


In [6]:
# Records that have a country
sightings_cleaned[sightings_cleaned.Country.isnull()==False]

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State
10,4/22/21 22:23,Fireball,2 minutes,Saw a steady pulsating fireball above that mov...,4/23/21,http://www.nuforc.org/webreports/162/S162818.html,\nSaw a steady pulsating fireball above that m...,saw steady pulsating fireball moved slowly awa...,New York City (Brooklyn),United States,New York,NY
11,4/22/21 21:00,Changing,6 minutes,Boomarang like shaped with five lights made a ...,4/23/21,http://www.nuforc.org/webreports/162/S162804.html,\nBoomarang like shaped with five lights made ...,boomarang like shaped five light made speed bi...,Firozabad (India),India,Firozabad,
24,4/20/21 20:30,Flash,3:39 minutes,"At approximately 8:30pm eastern time, I notice...",4/23/21,http://www.nuforc.org/webreports/162/S162797.html,"\nAt approximately 8:30pm eastern time, I noti...","approximately 8:30pm eastern time , noticed 1 ...",New York City (Brooklyn),United States,New York,NY
33,4/18/21 21:08,Light,15 seconds,Distant Satellite -Erratic Flight Path,4/23/21,http://www.nuforc.org/webreports/162/S162779.html,\nDistant Satellite -Erratic Flight PathAbout ...,distant satellite -erratic flight pathabout ho...,Nanaimo (Canada),Canada,Nanaimo,BC
95,4/10/21 19:00,Oval,3 seconds,"Mixed silver oval shape, sunlit reflexion cent...",4/23/21,http://www.nuforc.org/webreports/162/S162596.html,"\nMixed silver oval shape, sunlit reflexion ce...","mixed silver oval shape , sunlit reflexion cen...",Merseyside (UK/England),United Kingdom,Merseyside,
148,4/4/21 18:35,Circle,5 minutes,I saw a black steady object in the sky at appr...,4/23/21,http://www.nuforc.org/webreports/162/S162610.html,\nI saw a black steady object in the sky at ap...,saw black steady object sky approx 18:35. stay...,Melbourne (Australia),Australia,Melbourne,
152,4/3/21 05:30,Sphere,1 minute,Pulsating sphere interaction with a moving object,4/23/21,http://www.nuforc.org/webreports/162/S162633.html,\nPulsating sphere interaction with a moving o...,pulsating sphere interaction moving object5:00...,Winnipeg (Canada),Canada,Winnipeg,MB
203,3/26/21 05:07,Formation,1 minute,"Line of bright stars, moving rapidly across Ge...",3/31/21,http://www.nuforc.org/webreports/162/S162476.html,"\nLine of bright stars, moving rapidly across ...","line bright star , moving rapidly across germa...",Gilching (Germany),Germany,Gilching,
205,3/25/21 23:15,Light,30 seconds,Large bright white light in the night sky that...,3/31/21,http://www.nuforc.org/webreports/162/S162481.html,\nLarge bright white light in the night sky th...,large bright white light night sky suddenly va...,Chilliwack (Canada),Canada,Chilliwack,BC
215,3/25/21 21:00,Disk,5 minutes,6 lights surrounding a disc shap and moving,3/31/21,http://www.nuforc.org/webreports/162/S162492.html,\n6 lights surrounding a disc shap and movingT...,6 light surrounding disc shap movingthere appr...,Langford (Canada),Canada,Langford,BC


In [25]:
# Records that do not have a country
# sightings_cleaned[sightings_cleaned.Country.isnull()]
# sightings_cleaned.loc[(sightings_cleaned.Country.isnull())&(sightings_cleaned.State.isin(cities_df["State short"].unique()))]

60269

In [7]:
# Record without a country but have a state that is in the US state list
sightings_cleaned.loc[(sightings_cleaned.Country.isnull())&(sightings_cleaned.State.isin(cities_df["State short"].unique())), "Country"] = "United States"

In [8]:
# Record without a country but have a state that is NOT in the US state list
sightings_cleaned.loc[(sightings_cleaned.Country.isnull())&(sightings_cleaned.State.isin(cities_df["State short"].unique()==False))]

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State


In [9]:
cities_df.City.unique()

array(['Holtsville', 'Adjuntas', 'Aguada', ..., 'Point Baker',
       'Ward Cove', 'Wrangell'], dtype=object)

In [11]:
sightings_cleaned[(sightings_cleaned['City'].isin(cities_df["City alias"]))&(sightings_cleaned["Country"]=="United States")]

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State
0,4/23/21 06:30,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...,strangei recorded video sighting,NaN,United States,Blackshear,GA
1,4/23/21 06:00,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...,ball skyobject appears white ball vapor strewi...,NaN,United States,Mechanicsville,VA
2,4/23/21 06:00,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...,driving saw something strange sky pulled car i...,NaN,United States,Vero Beach,FL
4,4/23/21 05:58,Cone,>5 minutes,A cone of light coming from the sky unlike any...,4/23/21,http://www.nuforc.org/webreports/162/S162819.html,\nA cone of light coming from the sky unlike a...,cone light coming sky unlike anything ever see...,NaN,United States,Durham,NC
6,4/23/21 05:54,Light,5 minutes,Two bright lights one flashing with a descendi...,4/23/21,http://www.nuforc.org/webreports/162/S162820.html,\nTwo bright lights one flashing with a descen...,two bright light one flashing descending expan...,NaN,United States,Parrish,FL
7,4/23/21 05:45,Light,~10-15 minutes,Im former military and have never seen aircraf...,4/23/21,http://www.nuforc.org/webreports/162/S162826.html,\nIm former military and have never seen aircr...,I former military never seen aircraft that.inc...,NaN,United States,Champions Gate,FL
9,4/23/21 02:40,Chevron,3-4 seconds,"I witnessed a chevron-shaped object, silent an...",4/23/21,http://www.nuforc.org/webreports/162/S162827.html,"\nI witnessed a chevron-shaped object, silent ...","witnessed chevron-shaped object , silent seven...",NaN,United States,Firestone,CO
10,4/22/21 22:23,Fireball,2 minutes,Saw a steady pulsating fireball above that mov...,4/23/21,http://www.nuforc.org/webreports/162/S162818.html,\nSaw a steady pulsating fireball above that m...,saw steady pulsating fireball moved slowly awa...,New York City (Brooklyn),United States,New York,NY
12,4/22/21 20:50,Triangle,20+ minutes,metallic triangular shaped object appearing to...,4/23/21,http://www.nuforc.org/webreports/162/S162829.html,\nmetallic triangular shaped object appearing ...,metallic triangular shaped object appearing tu...,NaN,United States,Appleton,WI
13,4/22/21 20:42,Light,45 seconds,12 red lights in the sky low to the horizon in...,4/23/21,http://www.nuforc.org/webreports/162/S162817.html,\n12 red lights in the sky low to the horizon ...,"12 red light sky low horizon paulding county ,...",NaN,United States,Dallas,GA


In [12]:
sightings_cleaned.loc[(sightings_cleaned.City=="St. Augustine")&(sightings_cleaned.State=="FL"), "City"] = "Saint Augustine"

In [20]:
sightings_cleaned.loc[28, ["State", "Country"]] = ["PA", "United States"]

In [19]:
for index, row in sightings_cleaned[(sightings_cleaned['City'].isin(cities_df["City alias"]))&(sightings_cleaned["Country"]=="United States")].iterrows():
    sightings_cleaned.loc[index, "City"] = cities_df.loc[cities_df['City alias'] == row.City, "City"].tolist()[0]

KeyboardInterrupt: 

In [35]:
sightings_cleaned[(sightings_cleaned.Country.isnull())&(sightings_cleaned.Country.str.title().isin(countries.values()))]

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State


In [39]:
sightings_cleaned_bkup = sightings_cleaned.copy()

In [42]:
sightings_cleaned.loc[(sightings_cleaned.Country.isnull())&(sightings_cleaned.City.str.title().isin(countries.values())), "Country"] = sightings_cleaned.loc[(sightings_cleaned.Country.isnull())&(sightings_cleaned.City.str.title().isin(countries.values())), "City"]
sightings_cleaned.loc[(sightings_cleaned.Country.isnull())&(sightings_cleaned.City.str.title().isin(countries.values())), "City"] = ""

In [50]:
sightings_cleaned.loc[(sightings_cleaned.Country.isnull())&(sightings_cleaned.State.isnull()==False)&(sightings_cleaned.State!="")]

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State
131,4/6/21 21:00,Oval,8 minutes,Witnessed oval shaped light first appear as a ...,4/23/21,http://www.nuforc.org/webreports/162/S162551.html,\nWitnessed oval shaped light first appear as ...,witnessed oval shaped light first appear shoot...,NaN,NaN,Victoria,BC
492,2/17/21 14:45,Sphere,1:00,Quick mirror-like flashes from small object fo...,3/2/21,http://www.nuforc.org/webreports/162/S162070.html,\nQuick mirror-like flashes from small object ...,quick mirror-like flash small object following...,NaN,NaN,Embrun,ON
836,1/9/21 16:00,Other,20 minutes,Seen in sky floating upwards on January 9 at 4...,1/19/21,http://www.nuforc.org/webreports/161/S161578.html,\nSeen in sky floating upwards on January 9 at...,seen sky floating upwards january 9 4pm atlant...,NaN,NaN,Sydney,NS
945,12/31/20 21:00,Formation,3-5,Three Orange Lights Triangle,1/19/21,http://www.nuforc.org/webreports/161/S161467.html,\nThree Orange Lights TriangleThree orange lig...,three orange light trianglethree orange light ...,NaN,NaN,Edmonton,AB
960,12/29/20 23:00,Flash,Seconds to 1 Minute,4 Greenish Blue Blob-Like Objects Flashing Low...,1/19/21,http://www.nuforc.org/webreports/161/S161572.html,\n4 Greenish Blue Blob-Like Objects Flashing L...,4 greenish blue blob-like object flashing low ...,NaN,NaN,Belleville,ON
992,12/26/20 19:17,Unknown,,Yellow. illuminated odd shaped object.,1/19/21,http://www.nuforc.org/webreports/161/S161373.html,\nYellow. illuminated odd shaped object.I was ...,yellow . illuminated odd shaped object.i takin...,NaN,NaN,,BC
1048,12/20/20 22:30,Unknown,3 minutes,Looked out my kitchen window and noticed a bri...,12/23/20,http://www.nuforc.org/webreports/161/S161250.html,\nLooked out my kitchen window and noticed a b...,looked kitchen window noticed bright light clo...,NaN,NaN,Belleville,ON
1070,12/17/20 22:50,Light,2 seconds,Ball of white light travelling at incredible s...,12/23/20,http://www.nuforc.org/webreports/161/S161234.html,\nBall of white light travelling at incredible...,ball white light travelling incredible speed 3...,NaN,NaN,Pickering,ON
1073,12/17/20 18:30,Sphere,10 minutes,lights viewed,12/23/20,http://www.nuforc.org/webreports/161/S161228.html,\nlights viewed3 white lights with no sound. w...,light viewed3 white light sound . watched blin...,NaN,NaN,brampton,ON
1222,12/5/20 18:45,Formation,3- 5 minutes,8 lights in formation moving synchronously.,12/23/20,http://www.nuforc.org/webreports/160/S160972.html,\n8 lights in formation moving synchronously.C...,8 light formation moving synchronously.clear n...,NaN,NaN,Beaconsfield,QC


In [49]:
sightings_cleaned.loc[90626, "Country"] = "United Kingdom"

In [51]:
sightings_cleaned.loc[(sightings_cleaned.Country.isnull())&(sightings_cleaned.State.isnull()==False)&(sightings_cleaned.State!="")] = "Canada"

In [55]:
sightings_cleaned.to_pickle(cleaned_file_name)

In [54]:
sightings_cleaned.loc[(sightings_cleaned.Country.isnull())|((sightings_cleaned.State.isnull())&(sightings_cleaned.State==""))]

425